In [ ]:
!unzip Dataset

unzip:  cannot find or open Dataset, Dataset.zip or Dataset.ZIP.


In [11]:
!unzip Dataset.zip

Archive:  Dataset.zip
   creating: Dataset/.ipynb_checkpoints/
   creating: Dataset/Covid/
  inflating: Dataset/Covid/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg  
  inflating: Dataset/Covid/03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg  
  inflating: Dataset/Covid/0a7faa2a.jpg  
  inflating: Dataset/Covid/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-000-fig1a.png  
  inflating: Dataset/Covid/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-000-fig1b.png  
  inflating: Dataset/Covid/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-001-fig2a.png  
  inflating: Dataset/Covid/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-001-fig2b.png  
  inflating: Dataset/Covid/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-002-fig3a.png  
  inflating: Dataset/Covid/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-002-fig3b.png  
  inflating: Dataset/Covid/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-003-fig4a.png  
  inflating: Dataset/Covid/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-003-fig4b.png  
  inf

In [12]:
TRAIN_PATH = "Dataset/Train"

In [13]:
VAL_PATH = "Dataset/Test"

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [36]:
#CNN based model in keras
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 54, 54, 64)       

In [54]:
#Train from skratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [55]:
train_generator = train_datagen.flow_from_directory(
    'Dataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 338 images belonging to 2 classes.


In [56]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [57]:
validation_generator = test_dataset.flow_from_directory(
    'Dataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 338 images belonging to 3 classes.


In [58]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = 2
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 14s 2s/step - loss: 0.9905 - accuracy: 0.6446 - val_loss: 0.6324 - val_accuracy: 0.0312
Epoch 2/10
8/8 [==============================] - 12s 2s/step - loss: 0.5180 - accuracy: 0.7603 - val_loss: 0.5673 - val_accuracy: 0.0156
Epoch 3/10
8/8 [==============================] - 13s 2s/step - loss: 0.3464 - accuracy: 0.8633 - val_loss: 0.0306 - val_accuracy: 0.0469
Epoch 4/10
8/8 [==============================] - 13s 2s/step - loss: 0.2399 - accuracy: 0.9062 - val_loss: 1.1973 - val_accuracy: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.2091 - accuracy: 0.9215 - val_loss: 0.6175 - val_accuracy: 0.0156
Epoch 6/10
8/8 [==============================] - 12s 2s/step - loss: 0.2399 - accuracy: 0.9050 - val_loss: 0.8640 - val_accuracy: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 12s 2s/step - loss: 0.2061 - accuracy: 0.9380 - val_loss: 1.0308 - val_accuracy: 0.0000e+00
Epoch 8/10
8/8 [======

In [59]:
 model.save("model_adv.h5")

In [60]:
model.evaluate_generator(train_generator)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.12701623141765594, 0.9615384340286255]

In [61]:
model.evaluate(validation_generator)

11/11 [==============================] - 9s 791ms/step - loss: 0.9546 - accuracy: 0.0237


[0.9545621871948242, 0.023668639361858368]

In [62]:
model = load_model('model_adv.h5')

In [63]:
import os

In [64]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [65]:
y_actual = []
y_test = []

In [67]:
for i in os.listdir("./Dataset/Val/Normal/"):
  img = image.load_img("./Dataset/Val/Normal/"+i, target_size = (224,224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_generator(img)
  y_test.append(p[0,0])
  y_actual.append(1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [68]:
for i in os.listdir("./Dataset/Val/Covid/"):
  img = image.load_img("./Dataset/Val/Covid/"+i, target_size = (224,224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict(img)
  y_test.append(p[0,0])
  y_actual.append(0)

In [69]:
y_actual = np.array(y_actual)
y_test = np.array(y_test)

In [70]:
from sklearn.metrics import confusion_matrix

In [71]:
import seaborn as sns

In [72]:
from sklearn.metrics import classification_report

In [73]:
cm = confusion_matrix(y_actual, y_test)

ValueError: ignored